In [2]:
import geopandas as gpd
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
import folium

## Data processing

In [3]:
city = 'Luanda'

In [4]:
# read catchment AOI
aoi = gpd.read_file('AOI/luanda_catchment_level4.shp').to_crs(epsg = 4326)

In [5]:
output_folder = Path('output')
dam_data = gpd.read_file(r"C:\Users\Owner\Documents\Career\World Bank\CRP\data\GOODD\GOOD2_dams.shp")
reservoir_data = gpd.read_file(r"C:\Users\Owner\Documents\Career\World Bank\CRP\data\GRanD\GRanD_reservoirs_v1_3.shp")

In [6]:
dam_data.columns

Index(['DAM_ID', 'Count_ID', 'Latitud', 'Longitud', 'geometry'], dtype='object')

In [7]:
# filter for dams in AOI
dams_list = []

for i in range(len(aoi)):
    dams_list.append(dam_data.loc[dam_data.within(aoi.loc[i, 'geometry'])])

In [8]:
dams = pd.concat(dams_list)

In [8]:
dams.to_file(output_folder / f'{city.lower()}_dams.shp')

In [9]:
# filter for reservoirs in AOI
reservoirs_list = []

for i in range(len(aoi)):
    reservoirs_list.append(reservoir_data.loc[reservoir_data.within(aoi.loc[i, 'geometry'])])

In [10]:
reservoirs = pd.concat(reservoirs_list)

In [11]:
reservoirs.to_file(output_folder / f'{city.lower()}_reservoirs.shp')

## Visualization

In [11]:
dams['DAM_ID'] = dams['DAM_ID'].astype('int').astype('string')

In [12]:
reservoirs

,GRAND_ID,RES_NAME,DAM_NAME,ALT_NAME,RIVER,ALT_RIVER,MAIN_BASIN,SUB_BASIN,NEAR_CITY,ALT_CITY,...,MULTI_DAMS,TIMELINE,COMMENTS,URL,QUALITY,EDITOR,LONG_DD,LAT_DD,POLY_SRC,geometry
3672,3676,NaN,Quiminha,NaN,Bengo,NaN,South West Coast,Bengo,Catete,NaN,...,NaN,NaN,NaN,NaN,3: Fair,McGill-BL,13.790871,-8.963743,SWBD,"POLYGON ((14.04531 -8.89675, 14.04585 -8.89781..."


In [17]:
# Create a folium map centered on the AOI
mymap = folium.Map(control_scale = True)
bounds = aoi.total_bounds.tolist()
mymap.fit_bounds([bounds[:2][::-1], bounds[2:][::-1]])

folium.GeoJson(
    aoi,
    style_function=lambda feature: {
        'fillColor': 'transparent',
        'color': 'gray',
        'weight': 3
    },).add_to(mymap)

res_tooltip = folium.GeoJsonTooltip(fields=['RES_NAME'], aliases=['Reservoir Name'], localize=True, style=('background-color: white; color: #333333;'))
dam_tooltip = folium.GeoJsonTooltip(fields=['DAM_ID'], aliases=['Dam ID'], localize=True, style=('background-color: white; color: #333333;'))

# Add blue polygons for reservoirs
folium.GeoJson(
    reservoirs,
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 1,
    },
    highlight_function=lambda x: {"fillOpacity": 0.7},
    tooltip=res_tooltip,
    zoom_on_click=True
).add_to(mymap)

# Add CircleMarkers for dams
folium.GeoJson(
    dams,
    name="Dams",
    marker=folium.CircleMarker(radius=8, fill_color="brown", fill_opacity=0.4, color="brown", weight=1),
    tooltip = dam_tooltip,
    highlight_function=lambda x: {"fillOpacity": 0.7},
    zoom_on_click=True,
).add_to(mymap)

# Add legend manually
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 150px; height: 70px; 
                 background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
                 ">&nbsp; <b>Legend</b> <br>
                   &nbsp; Reservoirs &nbsp; 
                     <svg height="15" width="15">
                       <rect width="15" height="15" style="fill:blue;stroke:blue;stroke-width:2;fill-opacity:0.5" />
                     </svg><br>
                   &nbsp; Dams &nbsp; 
                     <svg height="10" width="10">
                       <circle cx="5" cy="5" r="5" style="fill:brown;stroke:brown;stroke-width:2;fill-opacity:0.5" />
                     </svg>
     </div>
     '''

mymap.get_root().html.add_child(folium.Element(legend_html))

mymap